# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500                                 #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv', 'MultiAgentIntersectionEnv_baseline_1', 'MultiAgentIntersectionEnv_baseline_2', 'MultiAgentIntersectionEnv_baseline_3', 'MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        # desired velocity for all vehicles in the network, in m/s
        "target_velocity": 30,
        # initial teamspirit
        "ap_teamspirit_0": -1,
        "ap_teamspirit_1": -1,
        # shuffle teamspirit?
        "ap_teamspirit_shuffle": False    
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-09_13-11-00_30240/logs.
Waiting for redis server at 127.0.0.1:24259 to respond...
Waiting for redis server at 127.0.0.1:62896 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=1c71038f05007dc71caeb68d84db6ade856d9270dd1ec1a0



{'node_ip_address': '172.16.123.117',
 'object_store_addresses': ['/tmp/ray/session_2019-04-09_13-11-00_30240/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-04-09_13-11-00_30240/sockets/raylet'],
 'redis_address': '172.16.123.117:24259',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=1c71038f05007dc71caeb68d84db6ade856d9270dd1ec1a0'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate default 0.999
config["model"].update({"fcnet_hiddens": [100, 50, 25]})  # size of hidden layers in network defaule 64 32
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return 'rl_0'

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn),
            'policies_to_train': ['rl_0']
        }
    })

 Starting SUMO on port 52973


New Teamspirit:
0.976419485470094
-0.4442784097254695


In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.0/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0_2019-04-09_13-11-02cdaml41i -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_13-11-57
  done: false
  episode_len_mean: 485.95
  episode_reward_max: 123.0041521056342
  episode_reward_mean: -8.499962313751194
  episode_reward_min: -200.90302216972958
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 6407.958
   

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=30302], 135 s, 5 iter, 50000 ts, -50.7 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_13-14-11
  done: false
  episode_len_mean: 297.34
  episode_reward_max: 157.88026843530776
  episode_reward_mean: -50.07914276241572
  episode_reward_min: -212.38164665177027
  episodes_this_iter: 41
  episodes_total: 167
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 4877.544
    load_time_ms: 10.471
    num_steps_sampled: 60000
    num_steps_trained: 60000
    rl_0:
      cur_kl_coeff: 0.012500002980232239
      cur_lr: 4.999999873689376e-05
      entropy: 1.380710482597351
      kl: 0.008414514362812042
     

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_13-16-21
  done: false
  episode_len_mean: 161.61
  episode_reward_max: 100.78826665440077
  episode_reward_mean: -52.440004880118934
  episode_reward_min: -208.23114062178718
  episodes_this_iter: 64
  episodes_total: 447
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 4445.614
    load_time_ms: 1.73
    num_steps_sampled: 110000
    num_steps_trained: 110000
    rl_0:
      cur_kl_coeff: 0.00039062509313225746
      cur_lr: 4.999999873689376e-05
      entropy: 1.3647818565368652
      kl: 0.00386752188205719
      policy_loss: -0.0005341600044630468
      total_loss: 734.5462646484375
      vf_explained_var: 0.3651774525642395
      vf_loss: 734.5467529296875
    sample_time_ms: 21887.172
    update_time_ms: 5.91
  iterations_since_restore: 11
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 30302
  policy_rewar

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=30302], 420 s, 16 iter, 160000 ts, -74.8 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_13-18-56
  done: false
  episode_len_mean: 131.66
  episode_reward_max: 70.42767800017589
  episode_reward_mean: -75.5627476129271
  episode_reward_min: -205.24244449241382
  episodes_this_iter: 76
  episodes_total: 862
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 4299.872
    load_time_ms: 1.774
    num_steps_sampled: 170000
    num_steps_trained: 170000
    rl_0:
      cur_kl_coeff: 6.103517080191523e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.3468657732009888
      kl: 0.004225405398756266
  

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_13-21-21
  done: false
  episode_len_mean: 117.63
  episode_reward_max: 62.73081598320505
  episode_reward_mean: -71.7889069665056
  episode_reward_min: -206.84527056337703
  episodes_this_iter: 81
  episodes_total: 1259
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 4631.72
    load_time_ms: 1.819
    num_steps_sampled: 220000
    num_steps_trained: 220000
    rl_0:
      cur_kl_coeff: 1.907349087559851e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.329553484916687
      kl: 0.0080961799249053
      policy_loss: -0.0028062909841537476
      total_loss: 991.7139282226562
      vf_explained_var: 0.45929795503616333
      vf_loss: 991.7167358398438
    sample_time_ms: 22530.555
    update_time_ms: 6.415
  iterations_since_restore: 22
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 30302
  policy_reward_m

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 11.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=30302], 20426 s, 693 iter, 6930000 ts, -103 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_18-53-03
  done: false
  episode_len_mean: 91.68468468468468
  episode_reward_max: 14.664200612591092
  episode_reward_mean: -77.70674096174987
  episode_reward_min: -200.67434800457954
  episodes_this_iter: 111
  episodes_total: 76931
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 6724.524
    load_time_ms: 1.951
    num_steps_sampled: 6940000
    num_steps_trained: 6940000
    rl_0:
      cur_kl_coeff: 1.3639874241513782e-35
      cur_lr: 4.999999873689376e-05
      entropy: -1.4222819805145264
      kl:

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 11.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=30302], 20595 s, 698 iter, 6980000 ts, -98.4 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_18-55-47
  done: false
  episode_len_mean: 83.94166666666666
  episode_reward_max: 11.966805737352818
  episode_reward_mean: -97.00392409937055
  episode_reward_min: -201.499038907765
  episodes_this_iter: 120
  episodes_total: 77513
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 5841.329
    load_time_ms: 1.742
    num_steps_sampled: 6990000
    num_steps_trained: 6990000
    rl_0:
      cur_kl_coeff: 1.0357780080605534e-34
      cur_lr: 4.999999873689376e-05
      entropy: -1.4507945775985718
      kl: 

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-04-09_18-58-14
  done: false
  episode_len_mean: 82.0
  episode_reward_max: 15.813648802308862
  episode_reward_mean: -100.81820880596023
  episode_reward_min: -200.76039289891713
  episodes_this_iter: 122
  episodes_total: 78096
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 5436.739
    load_time_ms: 1.568
    num_steps_sampled: 7040000
    num_steps_trained: 7040000
    rl_0:
      cur_kl_coeff: 5.243626373871348e-34
      cur_lr: 4.999999873689376e-05
      entropy: -1.4215832948684692
      kl: 0.04540468752384186
      policy_loss: 0.017488084733486176
      total_loss: 1283.5828857421875
      vf_explained_var: 0.5564661026000977
      vf_loss: 1283.5654296875
    sample_time_ms: 25585.456
    update_time_ms: 7.3
  iterations_since_restore: 704
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 30302
  policy_rewa

  custom_metrics: {}
  date: 2019-04-09_19-00-53
  done: false
  episode_len_mean: 84.21848739495799
  episode_reward_max: 14.567627296005941
  episode_reward_mean: -95.53286484493414
  episode_reward_min: -201.27183454439177
  episodes_this_iter: 119
  episodes_total: 78665
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 5744.334
    load_time_ms: 1.627
    num_steps_sampled: 7090000
    num_steps_trained: 7090000
    rl_0:
      cur_kl_coeff: 3.981878012841063e-33
      cur_lr: 4.999999873689376e-05
      entropy: -1.459372878074646
      kl: 0.06059017404913902
      policy_loss: 0.021716149523854256
      total_loss: 1457.4415283203125
      vf_explained_var: 0.49373334646224976
      vf_loss: 1457.419677734375
    sample_time_ms: 24811.588
    update_time_ms: 7.185
  iterations_since_restore: 709
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 30302
  policy_reward_mean:
    rl_0: -47.766432422467076
  time_since_res

  custom_metrics: {}
  date: 2019-04-09_19-03-44
  done: false
  episode_len_mean: 87.9375
  episode_reward_max: 18.120278824687297
  episode_reward_mean: -89.47321447914157
  episode_reward_min: -201.0405767598789
  episodes_this_iter: 112
  episodes_total: 79251
  experiment_id: 22b1cc9446844e61a011e26f0fdde742
  hostname: Gandalf
  info:
    grad_time_ms: 6053.825
    load_time_ms: 1.76
    num_steps_sampled: 7140000
    num_steps_trained: 7140000
    rl_0:
      cur_kl_coeff: 3.023739106173143e-32
      cur_lr: 4.999999873689376e-05
      entropy: -1.405792474746704
      kl: 11.528087615966797
      policy_loss: 0.17199382185935974
      total_loss: 1437.7547607421875
      vf_explained_var: 0.5020006895065308
      vf_loss: 1437.5828857421875
    sample_time_ms: 26837.189
    update_time_ms: 9.662
  iterations_since_restore: 714
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 30302
  policy_reward_mean:
    rl_0: -44.736607239570795
  time_since_restore: 21101.25